# file_system

In [ ]:
open sm'_operators

()



In [ ]:
// // test

open testing

()



## types

In [ ]:
inl types () =
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::path::Path\")>] type std_path_Path = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::path::PathBuf\")>] type std_path_PathBuf = class end"

()



In [ ]:
inl types () =
    global "#if FABLE_COMPILER // file_system.types"
    sm'.types ()
    rust.types ()
    types ()
    global "#endif // file_system.types"

()



## path

In [ ]:
nominal path = $'std_path_Path'
nominal path_buf = $'std_path_PathBuf'

()



## new_path_buf

In [ ]:
inl new_path_buf (path : string) : path_buf =
    open rust_operators
    inl path = path |> sm'.to_std_string
    !\\(path, $'"std::path::PathBuf::from($0)"')

()



## path_buf_from

In [ ]:
inl path_buf_from (path : rust.box path) : path_buf =
    open rust_operators
    !\\(path, $'"std::path::PathBuf::from($0)"')

()



## path_join

In [ ]:
inl path_join (s : string) (path_buf : path_buf) : path_buf =
    open rust_operators
    !\\((path_buf, s |> sm'.to_std_string), $'"$0.join($1)"')

()



## path_display

In [ ]:
inl path_display (path : rust.ref' path) : sm'.std_string =
    open rust_operators
    !\\(path, $'"$0.display().to_string()"')

()



## path_buf_file_name

In [ ]:
inl path_buf_file_name (path : path_buf) : optionm'.option' (rust.ref' sm'.os_str) =
    open rust_operators
    !\($'"!path.file_name()"')

()



## path_buf_display

In [ ]:
inl path_buf_display (path : path_buf) : sm'.std_string =
    open rust_operators
    !\\(path, $'"$0.display().to_string()"')

()



## path_exists

In [ ]:
inl path_buf_exists (path_buf : path_buf) : bool =
    open rust_operators
    !\($'"!path_buf.exists()"')

()



## path_is_dir

In [ ]:
inl path_buf_is_dir (path_buf : path_buf) : bool =
    open rust_operators
    !\($'"!path_buf.is_dir()"')

()



## path_is_file

In [ ]:
inl path_buf_is_file (path_buf : path_buf) : bool =
    open rust_operators
    !\($'"!path_buf.is_file()"')

()



## path_parent

In [ ]:
inl path_buf_parent (path_buf : path_buf) : optionm'.option' path_buf =
    open rust_operators
    !\\(path_buf, $'"$0.parent().map(std::path::PathBuf::from)"')

()



## ts_path_join

In [ ]:
inl ts_path_join (b : string) (a : string) : string =
    open ts_operators
    global "type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string"
    inl path : $'IPathJoin' = ts.import_all "path"
    !\\((a, b), $'"!path.join($0, $1)"')

()



## (< />)

In [ ]:
// let result = find_parent(".paket")(false)(get_source_directory());
// console.log(result);
inl (</>) (a : string) (b : string) : string =
    run_target function
        | Rust (Native) => fun () =>
            a |> new_path_buf |> path_join b |> path_buf_display |> sm'.from_std_string
        | Fsharp (Native) => fun () =>
            $'System.IO.Path.Combine (!a, !b)'
        | TypeScript _ => fun () =>
            a |> ts_path_join b
        | target => fun () => null ()

()



## get_temp_path

In [ ]:
inl get_temp_path () : string =
    $'System.IO.Path.GetTempPath' ()

()



## get_entry_assembly_name

In [ ]:
inl get_entry_assembly_name () : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.Reflection.Assembly.GetEntryAssembly().GetName().Name'
        | target => fun () => null ()

()



## create_temp_directory_name

In [ ]:
inl create_temp_directory_name () =
    run_target function
        | Fsharp (Native) => fun () =>
            get_temp_path ()
            </> ($'$"\!{!get_entry_assembly_name ()}"' : string)
            </> (date_time.now () |> date_time.new_guid_from_date_time |> sm'.obj_to_string)
        | target => fun () => null ()

()



In [ ]:
// // test

create_temp_directory_name ()
|> _contains ($'System.IO.Path.DirectorySeparatorChar' : char)

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
type [<Struct>] US0 =
    | US0_0 of f0_0 : char
    | US0_1
let rec closure0 () () : string =
    let v0 : string option = None
    let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v1 : string = null |> unbox<string>
    v1 
    #endif
    #if FABLE_COMPILER_RUST && WASM
    let v2 : string = null |> unbox<string>
    v2 
    #endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v3 : string = null |> unbox<string>
    v3 
    #endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !FABLE_COMPILER_TYPESCRIPT && !WASM && !CONTRACT
    let v4 : string = System.Reflection.Assembly.GetEntryAssembly().GetName().Name
    v4 
    #endif
    #if FABLE_COMPILER_TYPESCRIPT
    let v5 : string = null |> unbox<string>
    v5 
    #endif
    |> fun x -> _v0 <- Some x
    let v6 : string = _v0 |> Option.get
    v6
and method1 (v0 : string) : string =
    v0
and closure1 (v0 : char) (v1 : 

## directory_info

In [ ]:
nominal directory_info = $'System.IO.DirectoryInfo'

inl directory_info (path : string) : directory_info =
    path |> $'`directory_info '

()



## directory_info_exists

In [ ]:
inl directory_info_exists (info : directory_info) : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.Exists'
        | target => fun () => null ()

()



## directory_info_creation_time

In [ ]:
inl directory_info_creation_time (info : directory_info) : date_time.date_time =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.CreationTime'
        | target => fun () => null ()

()



## directory_info_name

In [ ]:
inl directory_info_name (info : directory_info) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.Name'
        | target => fun () => null ()

()



## directory_info_full_name

In [ ]:
inl directory_info_full_name (info : directory_info) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.FullName'
        | target => fun () => null ()

()



## create_directory

In [ ]:
inl create_directory (path : string) : directory_info =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Directory.CreateDirectory'
        | target => fun () => null ()

()



## directory_exists

In [ ]:
inl directory_exists (path : string) : bool =
    run_target function
        | Rust (Native) => fun () =>
            inl path = path |> new_path_buf
            (path |> path_buf_exists) && (path |> path_buf_is_dir)
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Directory.Exists'
        | TypeScript (Native) => fun () =>
            global "type IFsExistsSync = abstract existsSync: path: string -> bool"
            inl fs : $'IFsExistsSync' = ts.import_all "fs"
            ts.emit_expr path $'"!fs.existsSync($0)"'
        | target => fun () => null ()

()



## directory_get_parent

In [ ]:
inl directory_get_parent (path : string) : optionm'.option' string =
    run_target function
        | Rust (Native) => fun () =>
            path
            |> new_path_buf
            |> path_buf_parent
            |> optionm'.try'
            |> path_buf_display
            |> sm'.from_std_string
            |> Some
        | Fsharp (Native) => fun () =>
            inl parent : directory_info = path |> $'System.IO.Directory.GetParent'
            if parent =. null ()
            then None
            else parent |> directory_info_full_name |> Some
        | TypeScript _ => fun () =>
            global "type IPathDirname = abstract dirname: path: string -> string"
            inl fs : $'IPathDirname' = ts.import_all "path"
            ts.emit_expr path $'"!fs.dirname($0)"' |> Some
        | _ => fun () => null ()
        |> optionm'.box

()



## file_exists

In [ ]:
inl file_exists (path : string) : bool =
    run_target function
        | Rust (Native) => fun () =>
            inl path_buf = path |> new_path_buf
            (path_buf |> path_buf_exists) && (path_buf |> path_buf_is_file)
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.File.Exists'
        | TypeScript (Native) => fun () =>
            global "type IFsExistsSync = abstract existsSync: path: string -> bool"
            inl fs : $'IFsExistsSync' = ts.import_all "fs"
            ts.emit_expr path $'"!fs.existsSync($0)"'
        | target => fun () => null ()

()



## create_temp_directory

In [ ]:
inl create_temp_directory () =
    run_target function
        | Fsharp (Native) => fun () =>
            inl temp_folder = create_temp_directory_name ()
            inl result = temp_folder |> create_directory

            inl exists' = result |> directory_info_exists
            if not exists' then
                inl creation_time = result |> directory_info_creation_time
                inl result = ($'{| Exists = !exists'; CreationTime = !creation_time |}' : any) |> sm'.format_debug
                inl get_locals () : string =
                    $'$"temp_folder: {!temp_folder} / result: {!result} {!get_locals ()}"'
                // trace Debug (fun () => "createTempDirectory") get_locals
                console.write_line ($'$"create_temp_directory / {!get_locals ()}"' : string)

            temp_folder
        | target => fun () => null ()

()



In [ ]:
// // test

create_temp_directory ()
|> directory_exists
|> _assert_eq true

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
type IFsExistsSync = abstract existsSync: path: string -> bool
let rec closure0 () () : string =
    let v0 : string option = None
    let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v1 : string = null |> unbox<string>
    v1 
    #endif
    #if FABLE_COMPILER_RUST && WASM
    let v2 : string = null |> unbox<string>
    v2 
    #endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v3 : string = null |> unbox<string>
    v3 
    #endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !FABLE_COMPILER_TYPESCRIPT && !WASM && !CONTRACT
    let v4 : string = System.Reflection.Assembly.GetEntryAssembly().GetName().Name
    v4 
    #endif
    #if FABLE_COMPILER_TYPESCRIPT
    let v5 : string = null |> unbox<string>
    v5 
    #endif
    |> fun x -> _v0 <- Some x
    let v6 : string = _v0 |> Option.get
    v6
and method1 (v0 : string) : string =
    v0
and closure2 () () : string =

## get_source_directory

In [ ]:
inl get_source_directory () =
    $'__SOURCE_DIRECTORY__' : string

()



In [ ]:
// // test

get_source_directory ()
|> directory_info
|> directory_info_name
|> _assert_eq "spiral"

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = __SOURCE_DIRECTORY__
    let v1 : (string -> System.IO.DirectoryInfo) = System.IO.DirectoryInfo 
    let v2 : System.IO.DirectoryInfo = v1 v0
    let v3 : string option = None
    let mutable _v3 = v3
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v4 : string = null |> unbox<string>
    v4 
    #endif
    #if FABLE_COMPILER_RUST && WASM
    let v5 : string = null |> unbox<string>
    v5 
    #endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v6 : string = null |> unbox<string>
    v6 
    #endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !FABLE_COMPILER_TYPESCRIPT && !WASM && !CONTRACT
    let v7 : string = v2.Name
    v7 
    #endif
    #if FABLE_COMPILER_TYPESCRIPT
    let v8 : string = null |> unbox<string>
    v8 
    #endif
    |> fun x -> _v3 <- Some x
    let v9 : string = _v3 |> Option.get
    let v10 : string = $"%A{v9}"
    System.Console.WriteLine v10
    let v12 

## find_parent

In [ ]:
inl find_parent name is_file root_dir =
    let rec loop dir =
        if dir </> name |> (if is_file then file_exists else directory_exists)
        then dir
        else
            inl result = dir |> (join directory_get_parent)
            match result |> optionm'.unbox with
            | Some parent => parent |> loop
            | None => failwith $'$"""No parent for {if !is_file then "file" else "dir"} \'{!name}\' at \'{!root_dir}\'"""'
    loop root_dir

()



In [ ]:
// // test
// // print_code=false

a ;[ ".paket", false; "paket.dependencies", true ]
|> am.map fun (file, is_file) =>
    get_source_directory ()
    |> find_parent file is_file
    |> directory_info
    |> directory_info_name
|> am'.distinct
|> _assert_eq (a ;[ "polyglot" ] : _ i32 _)

[|"polyglot"|]


In [ ]:
// // test
// // rust=
// // print_code=false

types ()
a ;[ ".paket", false; "paket.dependencies", true ]
|> am.map fun (file, is_file) =>
    fun () => join
        get_source_directory ()
        |> find_parent file is_file
        |> new_path_buf
        |> path_buf_file_name
        |> optionm'.try'
        |> sm'.to_os_string
        |> sm'.os_to_str
        |> optionm'.try'
        |> sm'.ref_to_std_string
        |> sm'.from_std_string
        |> Some
        |> optionm'.box
    |> fun x => x () |> optionm'.unbox
    |> optionm'.default_value ""
|> am'.distinct
|> fun result =>
    result |> am'.length |> _assert_eq 1i32
    index result 0i32 |> _assert_eq "polyglot"

1
"polyglot"


## main

In [ ]:
inl main () =
    types ()
    $"let create_temp_directory_name () = !create_temp_directory_name ()" : ()
    $"let create_temp_directory () = !create_temp_directory ()" : ()
    $"let get_source_directory () = !get_source_directory ()" : ()
    $"let find_parent x = !find_parent x" : ()
    inl combine x = (</>) x
    $"let (</>) x = !combine x" : ()

#if FABLE_COMPILER // file_system.types
[<Fable.Core.Erase; Fable.Core.Emit("str")>] type Str = class end
[<Fable.Core.Erase; Fable.Core.Emit("base64::DecodeError")>] type base64_DecodeError = class end
[<Fable.Core.Erase; Fable.Core.Emit("borsh::io::Error")>] type borsh_io_Error = class end
[<Fable.Core.Erase; Fable.Core.Emit("js_sys::JsString")>] type js_sys_JsString = class end
[<Fable.Core.Erase; Fable.Core.Emit("serde_json::Error")>] type serde_json_Error = class end
[<Fable.Core.Erase; Fable.Core.Emit("serde_json::Value")>] type serde_json_Value = class end
[<Fable.Core.Erase; Fable.Core.Emit("serde_wasm_bindgen::Error")>] type serde_wasm_bindgen_Error = class end
[<Fable.Core.Erase; Fable.Core.Emit("std::ffi::OsStr")>] type std_ffi_OsStr = class end
[<Fable.Core.Erase; Fable.Core.Emit("std::ffi::OsString")>] type std_ffi_OsString = class end
[<Fable.Core.Erase; Fable.Core.Emit("std::str::Utf8Error")>] type std_str_Utf8Error = class end
[<Fable.Core.Erase; Fable.Core.Emit("std::s